In [1]:
import sys
sys.path.append('..')

In [2]:
from src.ARKAutoML import ARKAutoML
from sklearn.model_selection import train_test_split
from config import BaseConfig
import pandas as pd

In [3]:
df = pd.read_csv(BaseConfig.FILE_NAME)

In [4]:
train, valid = train_test_split(df)

In [5]:
aml = ARKAutoML(train,config=BaseConfig,
                   eval_metric='accuracy', 
                   n_trials=20, model_algos=['lgb'],
                   )

2021-09-24 08:33:14.800 | INFO     | src.ARKAutoML.ARKAutoML:add_project_config_to_logs:43 - 
{'BASE_INPUT_PATH': '../input/',
 'BASE_OUTPUT_PATH': '../output/',
 'DATA_PATH': '../input/../input/train.csv',
 'FILE_NAME': '../input/train.csv',
 'FOLD_METHOD': 'StratifiedKFold',
 'PROBLEM_TYPE': 'classification',
 'PROJECT_NAME': 'MySampleProject',
 'SUB_PROJECT_NAME': 'Sample_1',
 'TARGET_COL': 'customer_category',
 'cat_cols': ['customer_active_segment', 'X1'],
 'cont_cols': ['customer_visit_score',
               'customer_product_search_score',
               'customer_ctr_score',
               'customer_stay_score',
               'customer_frequency_score',
               'customer_product_variation_score',
               'customer_order_score',
               'customer_affinity_score']}
2021-09-24 08:33:14.807 | INFO     | src.ARKAutoML.create_folds:create_folds:26 - Fold 1 : 6442, 1611
2021-09-24 08:33:14.809 | INFO     | src.ARKAutoML.create_folds:create_folds:26 - Fold 2 : 644

In [6]:
aml.n_folds = 1

In [7]:
aml.create_optuna_optimization()

[I 2021-09-24 08:33:15,075] A new study created in memory with name: MySampleProject
C:\ProgramData\Anaconda3\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.1, 100] and q=1, but the range is not divisible by `q`. It will be replaced by [0.1, 99.1].
  warnings.warn(
2021-09-24 08:33:15.140 | INFO     | src.ARKAutoML.ARKAutoML:objective:79 - Trial No : 0, accuracy : 0.9758, Params : {'model_algo': 'lgb', 'num_leaves': 42, 'min_child_samples': 137, 'min_child_weight': 4, 'subsample': 1.0, 'colsample_bytree': 0.6, 'reg_alpha': 13.1, 'reg_lambda': 60.1}
                        Accuracy : 0.976  F1_score_0 : 0.986  F1_score_1 : 0.894  Precision_0 : 0.979  Precision_1 : 0.948  Recall_0 : 0.994  Recall_1 : 0.846  Roc_auc : 0.967
[I 2021-09-24 08:33:15,365] Trial 0 finished with value: 0.9757914338919925 and parameters: {'model_algo': 'lgb', 'num_leaves': 42, 'min_child_samples': 137, 'min_child_weight': 4, 'subsample': 1.0, 'colsample_bytree': 0

In [8]:
cross_validated_metrics, final_metrics_by_cut_off   = aml.get_crossvalidated_results_for_best_model(folds=5)

2021-09-24 08:33:17.860 | INFO     | src.ARKAutoML.ARKAutoML:get_crossvalidated_results_for_best_model:133 - Best Model : lgb, Model Params : {'num_leaves': 27, 'min_child_samples': 382, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_alpha': 2.1, 'reg_lambda': 60.1}
2021-09-24 08:33:17.897 | INFO     | src.ARKAutoML.ARKAutoML:get_crossvalidated_results_for_best_model:139 - Fold : 1, Train Shape : (6442, 10), Test Shape : (1611, 10)
2021-09-24 08:33:17.917 | INFO     | src.ARKAutoML.metrics:model_metrics:43 - Accuracy : 0.976  Precision_1 : 0.949  Recall_1 : 0.851  F1_score_1 : 0.897  Precision_0 : 0.98  Recall_0 : 0.994  F1_score_0 : 0.987  Roc_auc : 0.965
2021-09-24 08:33:17.920 | INFO     | src.ARKAutoML.metrics:model_metrics:44 - 
 [[1407    9]
 [  29  166]]
2021-09-24 08:33:17.926 | INFO     | src.ARKAutoML.metrics:model_metrics:45 - 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      1416
           1   

In [9]:
aml.train_best_model()

2021-09-24 08:33:18.579 | INFO     | src.ARKAutoML.ARKAutoML:train_best_model:190 - Training "lgb" Model with Best Params : {'num_leaves': 27, 'min_child_samples': 382, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_alpha': 2.1, 'reg_lambda': 60.1}
2021-09-24 08:33:18.654 | INFO     | src.ARKAutoML.metrics:model_metrics:43 - Accuracy : 0.975  Precision_1 : 0.944  Recall_1 : 0.844  F1_score_1 : 0.891  Precision_0 : 0.979  Recall_0 : 0.993  F1_score_0 : 0.986  Roc_auc : 0.981
2021-09-24 08:33:18.662 | INFO     | src.ARKAutoML.metrics:model_metrics:44 - 
 [[7031   49]
 [ 152  821]]
2021-09-24 08:33:18.675 | INFO     | src.ARKAutoML.metrics:model_metrics:45 - 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      7080
           1       0.94      0.84      0.89       973

    accuracy                           0.98      8053
   macro avg       0.96      0.92      0.94      8053
weighted avg       0.97      0.98     

In [10]:
score, metrics_by_cut_off = aml.score_validation_data(valid)

2021-09-24 08:33:18.744 | INFO     | src.ARKAutoML.ARKAutoML:score_validation_data:219 - Scoring Validation Data : Validation Shape : (2685, 10)
2021-09-24 08:33:18.777 | INFO     | src.ARKAutoML.metrics:model_metrics:43 - Accuracy : 0.978  Precision_1 : 0.961  Recall_1 : 0.851  F1_score_1 : 0.903  Precision_0 : 0.98  Recall_0 : 0.995  F1_score_0 : 0.988  Roc_auc : 0.983
2021-09-24 08:33:18.781 | INFO     | src.ARKAutoML.metrics:model_metrics:44 - 
 [[2352   11]
 [  48  274]]
2021-09-24 08:33:18.789 | INFO     | src.ARKAutoML.metrics:model_metrics:45 - 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      2363
           1       0.96      0.85      0.90       322

    accuracy                           0.98      2685
   macro avg       0.97      0.92      0.95      2685
weighted avg       0.98      0.98      0.98      2685

2021-09-24 08:33:18.863 | INFO     | src.ARKAutoML.ARKAutoML:score_validation_data:222 - Metrics by different Cu

In [11]:
aml.save_best_model()

2021-09-24 08:33:18.900 | INFO     | src.ARKAutoML.ARKAutoML:save_best_model:205 - ..\output\MySampleProject\Sample_1\models\best_model_preprocessor.pkl
2021-09-24 08:33:18.901 | INFO     | src.ARKAutoML.ARKAutoML:save_best_model:206 - ..\output\MySampleProject\Sample_1\models\best_model.pkl


In [12]:
test_d = valid.iloc[0:1, :].drop(BaseConfig.TARGET_COL,axis=1)

In [13]:
aml.predict(test_d)

(array([0], dtype=int8), array([[0.99568583, 0.00431417]]))

### Load PreTrained Model

In [14]:
aml2 = ARKAutoML(config=BaseConfig, loading=True)

2021-09-24 08:33:19.024 | INFO     | src.ARKAutoML.ARKAutoML:add_project_config_to_logs:43 - 
{'BASE_INPUT_PATH': '../input/',
 'BASE_OUTPUT_PATH': '../output/',
 'DATA_PATH': '../input/../input/train.csv',
 'FILE_NAME': '../input/train.csv',
 'FOLD_METHOD': 'StratifiedKFold',
 'PROBLEM_TYPE': 'classification',
 'PROJECT_NAME': 'MySampleProject',
 'SUB_PROJECT_NAME': 'Sample_1',
 'TARGET_COL': 'customer_category',
 'cat_cols': ['customer_active_segment', 'X1'],
 'cont_cols': ['customer_visit_score',
               'customer_product_search_score',
               'customer_ctr_score',
               'customer_stay_score',
               'customer_frequency_score',
               'customer_product_variation_score',
               'customer_order_score',
               'customer_affinity_score']}


In [15]:
aml2.load_best_model()

In [16]:
test_d = valid.iloc[0:1, :].drop(BaseConfig.TARGET_COL,axis=1)
aml2.predict(test_d)

(array([0], dtype=int8), array([[0.99568583, 0.00431417]]))

### Only Train on entire Data and Test on Valid

This method is not recommended. But can be used in case of low resources availability.

In [17]:
aml = ARKAutoML(train,
                   eval_metric='accuracy', 
                   n_folds=False, n_trials = 1, 
                   model_algos=['rf'],
                   config=BaseConfig)

2021-09-24 08:33:19.167 | INFO     | src.ARKAutoML.ARKAutoML:add_project_config_to_logs:43 - 
{'BASE_INPUT_PATH': '../input/',
 'BASE_OUTPUT_PATH': '../output/',
 'DATA_PATH': '../input/../input/train.csv',
 'FILE_NAME': '../input/train.csv',
 'FOLD_METHOD': 'StratifiedKFold',
 'PROBLEM_TYPE': 'classification',
 'PROJECT_NAME': 'MySampleProject',
 'SUB_PROJECT_NAME': 'Sample_1',
 'TARGET_COL': 'customer_category',
 'cat_cols': ['customer_active_segment', 'X1'],
 'cont_cols': ['customer_visit_score',
               'customer_product_search_score',
               'customer_ctr_score',
               'customer_stay_score',
               'customer_frequency_score',
               'customer_product_variation_score',
               'customer_order_score',
               'customer_affinity_score']}


In [18]:
aml.create_only_one_train_and_one_valid(valid)

In [19]:
aml.create_optuna_optimization()

[I 2021-09-24 08:33:19,290] A new study created in memory with name: MySampleProject
2021-09-24 08:33:19.971 | INFO     | src.ARKAutoML.ARKAutoML:objective:79 - Trial No : 0, accuracy : 0.9769, Params : {'model_algo': 'rf', 'max_depth': 5, 'n_estimators': 348, 'min_samples_split': 58, 'min_samples_leaf': 21}
                        Accuracy : 0.977  F1_score_0 : 0.987  F1_score_1 : 0.898  Precision_0 : 0.979  Precision_1 : 0.958  Recall_0 : 0.995  Recall_1 : 0.845  Roc_auc : 0.98
[I 2021-09-24 08:33:20,015] Trial 0 finished with value: 0.9769087523277468 and parameters: {'model_algo': 'rf', 'max_depth': 5, 'n_estimators': 348, 'min_samples_split': 58, 'min_samples_leaf': 21}. Best is trial 0 with value: 0.9769087523277468.
